# Feature Importance Methods for Scientific Inference

In these exercises you will:

1. Interpret a **Permutation Feature Importance (PFI)** plot and reflect on what PFI measures
2. Discover **why PFI can be misleading** when features are correlated — and implement PFI yourself
3. Use a **conditional sampler** to compute **Conditional Feature Importance (CFI)** and compare
4. Implement **Leave-One-Covariate-Out (LOCO)** importance based on conditional marginalization

---

## Setup

Run the cells below to set everything up. **You do not need to modify any code in this section.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'font.size':        14,
    'axes.titlesize':   16,
    'axes.labelsize':   14,
    'xtick.labelsize':  13,
    'ytick.labelsize':  13,
    'legend.fontsize':  13,
})

### Data Generation

We generate a dataset with 5 features and a continuous target variable $Y$.

**You do not know the data generating process (DGP) yet.** You only observe the features $X_1, X_2, X_3, X_4, X_5$ and the target $Y$.

In [ ]:
def generate_data(n=1500, seed=83):
    """Generate the dataset. The DGP is hidden for now."""
    rng = np.random.RandomState(seed)
    x1 = rng.normal(0, 1, n)
    x2 = 0.999 * x1 + np.sqrt(1 - 0.999**2) * rng.normal(0, 1, n)
    x3 = rng.normal(0, 1, n)
    x4 = 0.999 * x3 + np.sqrt(1 - 0.999**2) * rng.normal(0, 1, n)
    y = 5 * x1 + rng.normal(0, 1, n)
    x5 = rng.normal(0, 1, n)
    X = np.column_stack([x1, x2, x3, x4, x5])
    return X, y

X, y = generate_data()
feature_names = ["X1", "X2", "X3", "X4", "X5"]

print(f"Dataset shape: {X.shape}")
print(f"Feature names: {feature_names}")
print(f"\nFirst 5 rows of X:")
print(np.round(X[:5], 2))
print(f"\nFirst 5 values of Y: {np.round(y[:5], 2)}")

### Train a model

We train a Linear Regression on the data.

In [ ]:
# Split into train and test
n_train = 1000
X_train, X_test = X[:n_train], X[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

# Train a Linear Regression (unregularized)
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = model.score(X_test, y_test)
print(f"Test MSE: {mse:.3f}")
print(f"Test R\u00b2:  {r2:.3f}")
print(f"\nFitted coefficients: {np.round(model.coef_, 2)}")

### Provided: Permutation Sampler and PFI

Below we provide:
- A **permutation sampler** that randomly shuffles $X_j$, breaking its association with all other variables.
- A **`compute_pfi` function** that computes PFI by averaging the loss increase over many permutations.

$$\text{PFI}_j = \mathbb{E}[L(Y, \hat{f}(\tilde{X}_j, X_{-j}))] - \mathbb{E}[L(Y, \hat{f}(X))]$$

where $\tilde{X}_j \sim P(X_j)$ is drawn independently of everything else.

In [ ]:
def permutation_sampler(X, feature_idx, rng=None):
    """
    Permutation (marginal) sampler.
    Returns a copy of X where column `feature_idx` is randomly permuted.
    """
    if rng is None:
        rng = np.random.RandomState(0)
    X_perm = X.copy()
    X_perm[:, feature_idx] = rng.permutation(X[:, feature_idx])
    return X_perm


def compute_pfi(model, X, y, feature_idx, sampler, n_repeats=50, seed=42):
    """
    Compute Permutation Feature Importance for feature `feature_idx`.
    Returns the mean increase in MSE over n_repeats permutations.
    """
    rng = np.random.RandomState(seed)
    baseline_mse = mean_squared_error(y, model.predict(X))
    perturbed_mses = []
    for _ in range(n_repeats):
        X_perturbed = sampler(X, feature_idx, rng=rng)
        perturbed_mses.append(mean_squared_error(y, model.predict(X_perturbed)))
    return np.mean(perturbed_mses) - baseline_mse

### Provided: PFI Results

The cell below computes PFI for all features and shows the bar chart. **You do not need to modify this cell.**

In [ ]:
# Pre-computed PFI results (provided)
pfi_scores = [compute_pfi(model, X_test, y_test, feature_idx=j,
                          sampler=permutation_sampler)
              for j in range(X_test.shape[1])]

plt.figure(figsize=(6, 4))
plt.barh(feature_names[::-1], pfi_scores[::-1],
         color='grey', edgecolor='black', linewidth=0.5)
plt.xlabel("PFI (increase in MSE)")
plt.title("Permutation Feature Importance")
plt.tight_layout()
plt.show()

---

# Exercise 1: Interpret the PFI Plot

Look at the PFI bar chart above. The model achieves $R^2 = 0.963$ on the test set.

**Questions:**

1. Which features does the **model** rely on for its predictions?
2. Based on PFI alone, which features would you conclude are **important in the data** (i.e., associated with $Y$)?
3. What exactly does PFI measure?

**Your interpretation (double-click to edit):**

- Which features does the model rely on?
  - *Your answer here*

- Which features appear important in the data?
  - *Your answer here*

- What does PFI measure exactly?
  - *Your answer here*

---

# Exercise 2: Why is PFI Misleading? Implement it Yourself.

Now we reveal the **true data generating process (DGP)**:

$$X_1 \sim \mathcal{N}(0,1), \quad X_3 \sim \mathcal{N}(0,1), \quad X_5 \sim \mathcal{N}(0,1) \quad \text{(mutually independent)}$$

$$X_2 = 0.999 \cdot X_1 + \sqrt{1 - 0.999^2} \cdot \varepsilon_2, \quad X_4 = 0.999 \cdot X_3 + \sqrt{1 - 0.999^2} \cdot \varepsilon_4$$

$$Y = 5 X_1 + \varepsilon_Y, \quad \varepsilon_Y \sim \mathcal{N}(0, 1)$$

The fitted model is:
$$\hat{f}(X) = 3.11\,X_1 + 1.88\,X_2 - 2.11\,X_3 + 2.17\,X_4 + 0.02\,X_5$$

Were your interpretations from Exercise 1 correct?

**Tasks:**

1. **Implement PFI from scratch.** Write `my_pfi(model, X, y, feature_idx)` that permutes feature `feature_idx` once and measures the MSE increase. Average over 50 repetitions. Verify it matches the provided `compute_pfi`.
2. **Create scatterplots** of $(X_3, X_4)$ before and after permuting $X_3$. What do you notice?
3. **Explain** why PFI assigns high importance to $X_3$ and $X_4$ even though they are completely independent of $Y$.

In [ ]:
# Task 1: Implement PFI from scratch
def my_pfi(model, X, y, feature_idx, n_repeats=50, seed=42):
    """
    Compute PFI for a single feature by permuting it n_repeats times.
    Returns the mean increase in MSE.
    """
    pass  # replace with your implementation


# Verify against the provided compute_pfi
for j in range(X_test.shape[1]):
    mine     = my_pfi(model, X_test, y_test, feature_idx=j)
    provided = pfi_scores[j]
    print(f"X{j+1}: mine={mine:.4f}  provided={provided:.4f}")

In [ ]:
# Task 2: Scatterplots of (X3, X4) before and after permuting X3
rng = np.random.RandomState(42)
X_test_perm_x3 = permutation_sampler(X_test, feature_idx=2, rng=rng)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

axes[0].scatter(X_test[:, 2], X_test[:, 3], alpha=0.3, s=10, color='grey')
axes[0].set_xlabel("$X_3$"); axes[0].set_ylabel("$X_4$")
axes[0].set_title("Original: $(X_3, X_4)$")
axes[0].set_xlim(-4, 4); axes[0].set_ylim(-4, 4)

axes[1].scatter(X_test_perm_x3[:, 2], X_test_perm_x3[:, 3], alpha=0.3, s=10, color='grey')
axes[1].set_xlabel(r"$\tilde{X}_3$ (permuted)"); axes[1].set_ylabel("$X_4$")
axes[1].set_title("After permuting $X_3$")
axes[1].set_xlim(-4, 4); axes[1].set_ylim(-4, 4)

plt.tight_layout()
plt.show()

**Your explanation (double-click to edit):**

- What do you notice in the scatterplots?
  - *Your answer here*

- How do the model coefficients explain the PFI scores?
  - *Your answer here*

- Why does PFI assign high importance to $X_3$ and $X_4$?
  - *Your answer here*

---

# Exercise 3: Conditional Feature Importance (CFI)

To address the problem from Exercise 2, we use a **conditional sampler** instead of the permutation sampler.

Instead of sampling $\tilde{X}_j$ from the marginal (breaking all dependencies), we sample from:

$$\tilde{X}_j \sim P(X_j \mid X_{-j})$$

This preserves the dependencies between features while still breaking the direct $X_j$–$Y$ link.

For multivariate normal data this has a closed form:

$$X_j \mid X_{-j} = x_{-j} \sim \mathcal{N}\!\left(\mu_j + \Sigma_{j,-j}\,\Sigma_{-j,-j}^{-1}(x_{-j} - \mu_{-j}),\; \Sigma_{jj} - \Sigma_{j,-j}\,\Sigma_{-j,-j}^{-1}\,\Sigma_{-j,j}\right)$$

The conditional sampler and wrapper are provided below.

In [ ]:
def conditional_sampler(X, feature_idx, rng=None, mean=None, cov=None):
    """
    Conditional sampler for multivariate normal data.
    Samples X_j from X_j | X_{-j} using the closed-form Gaussian conditional.
    """
    if rng is None:
        rng = np.random.RandomState(0)
    n, p = X.shape
    j = feature_idx
    others = [i for i in range(p) if i != j]

    sigma_jj           = cov[j, j]
    sigma_j_others     = cov[j, others]
    sigma_others_others = cov[np.ix_(others, others)]
    sigma_others_inv   = np.linalg.inv(sigma_others_others)
    beta               = sigma_j_others @ sigma_others_inv
    cond_var           = sigma_jj - sigma_j_others @ sigma_others_inv @ sigma_j_others

    x_others   = X[:, others]
    cond_means = mean[j] + (x_others - mean[others]) @ beta
    X_cond     = X.copy()
    X_cond[:, j] = cond_means + rng.normal(0, np.sqrt(max(cond_var, 0)), n)
    return X_cond


# Estimate mean and covariance from training data
estimated_mean = np.mean(X_train, axis=0)
estimated_cov  = np.cov(X_train, rowvar=False)

print("Estimated covariance matrix:")
print(np.round(estimated_cov, 3))

def conditional_sampler_wrapper(X, feature_idx, rng=None):
    return conditional_sampler(X, feature_idx, rng=rng,
                               mean=estimated_mean, cov=estimated_cov)

**Tasks:**

1. Compute **CFI** for all features using `compute_pfi` with the `conditional_sampler_wrapper`.
2. Create a side-by-side bar chart comparing PFI and CFI.
3. Interpret the results:
   - How do PFI and CFI differ? Why?
   - How does CFI treat each type of feature ($X_1$ directly relevant, $X_2$ indirectly relevant, $X_3$/$X_4$ irrelevant-collinear, $X_5$ purely irrelevant)?
   - What does this mean for scientific inference?

In [ ]:
# Task 1: Compute CFI for all features
cfi_scores = []
for j in range(X_test.shape[1]):
    cfi_scores.append(compute_pfi(model, X_test, y_test, feature_idx=j,
                                  sampler=conditional_sampler_wrapper))

# Task 2: Side-by-side bar chart
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
names_r = feature_names[::-1]

axes[0].barh(names_r, pfi_scores[::-1], color='grey', edgecolor='black', linewidth=0.5)
axes[0].set_xlabel("Importance (increase in MSE)")
axes[0].set_title("PFI (Permutation / Marginal Sampler)")

axes[1].barh(names_r, cfi_scores[::-1], color='grey', edgecolor='black', linewidth=0.5)
axes[1].set_xlabel("Importance (increase in MSE)")
axes[1].set_title("CFI (Conditional Sampler)")

plt.tight_layout()
plt.show()

**Your interpretation (double-click to edit):**

- How do PFI and CFI differ? Why?
  - *Your answer here*

- How does CFI treat each type of feature?
  - *Your answer here*

- What does this mean for scientific inference?
  - *Your answer here*

---

# Exercise 4: Leave-One-Covariate-Out (LOCO)

CFI asks: *how much does performance drop when we remove the association between $X_j$ and $Y$?*
**LOCO** asks: *how much does performance drop when we remove feature $j$ from the model entirely?*

## The conditional SAGE value function

For a coalition (subset) $S \subseteq \{1,\ldots,p\}$, define:

$$v(S) = \underbrace{\mathbb{E}\!\left[(Y - \mathbb{E}[f(X)])^2\right]}_{\text{baseline MSE}} - \underbrace{\mathbb{E}\!\left[(Y - \mathbb{E}[f(X)\mid X_S])^2\right]}_{\text{MSE using only } X_S}$$

$v(S)$ measures how much explained variance is gained by using only the features in $S$.
Note $v(\emptyset) = 0$ and $v(\{1,\ldots,p\}) \approx \text{Var}(Y) \cdot R^2$.

For a **linear model** $f(X) = \beta_0 + \beta^\top X$ with **Gaussian features**:

$$\mathbb{E}[f(X) \mid X_S] = \beta_0 + \beta_S^\top X_S + \beta_{S^c}^\top\!\left(\mu_{S^c} + \Sigma_{S^c,S}\,\Sigma_{S,S}^{-1}(X_S - \mu_S)\right)$$

## LOCO from the value function

$$\text{LOCO}_j = v(\{1,\ldots,p\}) - v(\{1,\ldots,p\} \setminus \{j\})$$

This is the drop in explained variance when feature $j$ is removed from the full set.

---

**Tasks:**

1. Implement `conditional_sage_value(S, model, X, y, mean, cov)` that computes $v(S)$ using the closed-form Gaussian conditional for a linear model.
2. Compute **LOCO** for all features as $v(N) - v(N \setminus \{j\})$.
3. Normalise by the baseline MSE to express each score as a **share of explained variance (%)**.
4. Plot the results (horizontal grey bars, $X_1$ on top) and interpret.

In [ ]:
# Task 1: Implement the conditional SAGE value function
def conditional_sage_value(S, model, X, y, mean, cov):
    """
    Compute v(S) = MSE_base - E[(Y - E[f(X)|X_S])^2].

    For a linear model with Gaussian features, E[f(X)|X_S] is:
      beta0 + beta_S @ X_S + beta_Sc @ (mean_Sc + Sigma_{Sc,S} @ inv(Sigma_{S,S}) @ (X_S - mean_S))

    Parameters
    ----------
    S    : tuple of int  – feature indices in the coalition
    model: fitted LinearRegression
    X    : np.ndarray (n, p)
    y    : np.ndarray (n,)
    mean : np.ndarray (p,)  – estimated feature means
    cov  : np.ndarray (p,p) – estimated feature covariance

    Returns
    -------
    float – v(S)
    """
    pass  # replace with your implementation

In [ ]:
# Task 2 & 3: Compute LOCO scores and normalise
p = X_test.shape[1]
N = tuple(range(p))

loco_scores_ex4 = []
for j in range(p):
    N_minus_j = tuple(i for i in range(p) if i != j)
    loco_j = conditional_sage_value(N, model, X_test, y_test, estimated_mean, estimated_cov) \
           - conditional_sage_value(N_minus_j, model, X_test, y_test, estimated_mean, estimated_cov)
    loco_scores_ex4.append(loco_j)

v_total = conditional_sage_value(N, model, X_test, y_test, estimated_mean, estimated_cov)

for j in range(p):
    share = 100 * loco_scores_ex4[j] / v_total
    print(f"LOCO({feature_names[j]}): {loco_scores_ex4[j]:.4f}  ({share:.1f}% of explained variance)")

In [ ]:
# Task 4: Plot LOCO scores
v_total_plot = conditional_sage_value(N, model, X_test, y_test, estimated_mean, estimated_cov)
loco_pct = [100 * s / v_total_plot for s in loco_scores_ex4]

plt.figure(figsize=(6, 4))
plt.barh(feature_names[::-1], loco_pct[::-1],
         color='grey', edgecolor='black', linewidth=0.5)
plt.xlabel("Share of explained variance (%)")
plt.title("LOCO (conditional marginalization)")
plt.axvline(0, color='black', linewidth=0.8)
plt.tight_layout()
plt.show()

**Your interpretation (double-click to edit):**

- Which features have non-zero LOCO values? Why?
  - *Your answer here*

- What fraction of the explained variance ($R^2$) is attributed to each feature?
  - *Your answer here*

- How do the LOCO scores compare to the CFI scores from Exercise 3?
  - *Your answer here*

- What does LOCO measure that CFI does not?
  - *Your answer here*